<a href="https://colab.research.google.com/github/roamingleaf/test-20181028/blob/master/My_FuDan_Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored

In [ ]:
 !pip install -q -U tensorflow-text

In [ ]:
!pip install -q -U tf-models-official

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

tf.get_logger().setLevel('ERROR')

print(os.environ['COLAB_TPU_ADDR'])



10.5.217.90:8470


In [ ]:
bert_model_name = 'bert_en_uncased_L-12_H-768_A-12' 
bert_handle_encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
bert_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def make_bert_preprocess_model(sentence_features, seq_length=512):
  inputs=[tf.keras.layers.Input(shape=(),dtype=tf.string,name=ft) for ft in sentence_features]
  bert_preprocess = hub.load(bert_handle_preprocess)
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in inputs]
  '''
  Each preprocessing model also provides a method, .bert_pack_inputs(tensors, seq_length)
  input_word_ids:
  input_mask    : contains a 1 anywhere the input_word_ids is not padding.
  input_type_ids: contains a 0 or a 1 indicating which sentence the token is a part of.
  '''
  packer=hub.KerasLayer(bert_preprocess.bert_pack_inputs,name='pakcer')
  outputs=packer(segments)
  '''
  Returns:
  A Keras Model that can be called on a list or dict of string Tensors
  (with the order or names, resp., given by sentence_features) and
  returns a dict of tensors for input to BERT.
  """
  '''
  return tf.keras.Model(inputs,outputs)


AUTOTUNE=tf.data.AUTOTUNE
def load_dataset_from_file(file_name,batch_size,bert_preprocess_model,label_column='star',is_training=True,): 
  dataset = tf.data.experimental.make_csv_dataset(
                    file_name, batch_size=batch_size, label_name=label_column)
  if is_training:
    dataset = dataset.shuffle(1000).repeat(2)
  dataset = dataset.map(lambda ex: (bert_preprocess_model(ex), ex[label_column]))
  dataset=dataset.cache().prefectch(buffer_size=AUTOTUNE)
  return dataset
  #iterator = ds.as_numpy_iterator()


def build_classifier_model(num_classes):
  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer(bert_handle_encoder, trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.1)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [ ]:
def get_configuration(num_classes):
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  metrics = tfa.metrics.MatthewsCorrelationCoefficient(num_classes=5)
  return metrics, loss


epochs = 3
batch_size = 32
init_lr = 2e-5
num_classes=5
sentence_features=(['my_sentence'])
filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')

print(f'Fine tuning {bert_handle_encoder} model')

bert_preprocess_model = make_bert_preprocess_model(sentence_features)


Fine tuning https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3 model


In [ ]:

#bert_preprocess_model = make_bert_preprocess_model(sentence_features)
dataset = tf.data.experimental.make_csv_dataset(filepath_dev, batch_size=batch_size, label_name='star')
print(dataset.take(1))
#iterator = dataset.as_numpy_iterator()
#print(dict(next(iterator)))

dataset = dataset.map(lambda _,ex: (bert_preprocess_model(ex), ex[1]))
print(dataset)
#dataset = dataset.map(lambda _,ex:(_,ex))
#dataset=dataset.cache().prefetch(buffer_size=AUTOTUNE)


<TakeDataset shapes: (OrderedDict([(\ufeffid, (32,)), (review, (32,)), (Location#Transportation, (32,)), (Location#Downtown, (32,)), (Location#Easy_to_find, (32,)), (Service#Queue, (32,)), (Service#Hospitality, (32,)), (Service#Parking, (32,)), (Service#Timely, (32,)), (Price#Level, (32,)), (Price#Cost_effective, (32,)), (Price#Discount, (32,)), (Ambience#Decoration, (32,)), (Ambience#Noise, (32,)), (Ambience#Space, (32,)), (Ambience#Sanitary, (32,)), (Food#Portion, (32,)), (Food#Taste, (32,)), (Food#Appearance, (32,)), (Food#Recommend, (32,))]), (32,)), types: (OrderedDict([(\ufeffid, tf.int32), (review, tf.string), (Location#Transportation, tf.int32), (Location#Downtown, tf.int32), (Location#Easy_to_find, tf.int32), (Service#Queue, tf.int32), (Service#Hospitality, tf.int32), (Service#Parking, tf.int32), (Service#Timely, tf.int32), (Price#Level, tf.int32), (Price#Cost_effective, tf.int32), (Price#Discount, tf.int32), (Ambience#Decoration, tf.int32), (Ambience#Noise, tf.int32), (Ambien

In [ ]:

filepath='/content/Fudan/data/meituan'
## Read file to pd data
filepath_dev=os.path.join(filepath,'dev.csv')
filepath_test=os.path.join(filepath,'test.csv')
metrics, loss = get_configuration(num_classes)

train_dataset = load_dataset_from_file(
      filepath_dev, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
train_data_size=validation_data_size=4900
steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10

validation_dataset= load_dataset_from_file(
      filepath_test, batch_size=512,bert_preprocess_model=bert_preprocess_model)
  #########
validation_steps = validation_data_size // batch_size

classifier_model = build_classifier_model(num_classes)

optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')

classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])

classifier_model.fit(
      x=train_dataset,
      validation_data=validation_dataset,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_steps=validation_steps)

TypeError: ignored